In [19]:
import fitz  # PyMuPDF
import requests
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 로드
API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}
#  요약 모델: BART (영어 요약)
SUMMARIZATION_API = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

#  번역 모델 (영어 → 한국어 번역)
TRANSLATION_API = "https://api-inference.huggingface.co/models/facebook/nllb-200-distilled-600M"

# PDF 파일에서 텍스트 추출 
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text
    
#논문 내용을 영어로 요약
def summarize_text(text):
    if not text.strip():
        return "입력된 텍스트가 없습니다."
    
    prompt = f"""Summarize the following research paper **in detail** while keeping a structured format. Ensure the summary captures all essential aspects concisely.

### **Title & Authors**
- Mention the title and main authors.

### **Research Objective**
- What is the research about? What problem does it solve?

### **Methodology**
- Describe the approach, models, datasets, and key techniques used.

### **Key Findings & Performance**
- Summarize the main results and compare them to previous methods.
- Provide key performance metrics.

### **Conclusion & Future Work**
- What are the key takeaways and potential research directions?

**Paper Content:**  
{text[:3000]}""" 
    
    payload = {"inputs": prompt, "parameters": {"max_new_tokens": 300, "temperature": 0.7}}
    
    response = requests.post(SUMMARIZATION_API, headers=HEADERS, json=payload)
    result = response.json()

    print(" 요약 결과 (영어):", result)  # API 응답 확인
    
    if response.status_code == 200:
        if isinstance(result, list) and 'summary_text' in result[0]:  # BART 모델 응답 처리
            return result[0]['summary_text']
        else:
            return f"예상치 못한 응답 형식: {result}"
    else:
        return f"API 오류: {response.status_code}, {result}"


#영어 요약 내용을 한국어로 번역 (언어 코드 사용)
def translate_to_korean(english_text):
    payload = {
        "inputs": english_text,
        "parameters": {
            "max_new_tokens": 250,  # 250 이하로 설정
            "src_lang": "eng_Latn",  # 원본 언어: 영어
            "tgt_lang": "kor_Hang",  # 목표 언어: 한국어
        }
    }
    
    response = requests.post(TRANSLATION_API, headers=HEADERS, json=payload)
    
    print(f"번역 API 응답 상태 코드: {response.status_code}")
    print(f"번역 API 응답 텍스트: {response.text}")

    if response.status_code != 200 or not response.text.strip():
        return f"PI 오류: {response.status_code}, 응답이 비어 있음"

    try:
        result = response.json()
    except requests.exceptions.JSONDecodeError:
        return f"JSONDecodeError: API 응답을 디코딩할 수 없음. 응답 내용: {response.text}"

    if isinstance(result, list) and 'translation_text' in result[0]:
        return result[0]['translation_text']
    else:
        return f"번역 실패: {result}"

# 실행 코드 (로컬 PDF 파일 입력)
pdf_path = "/Users/anchanghun/Downloads/Document_ach/paper/9_AnomalyDetection/TSSAN_Time-Space_Separable_Attention_Network_for_Intrusion_Detection.pdf"
text = extract_text_from_pdf(pdf_path)

# 논문 요약 (영어)
summary = summarize_text(text)

# 영어 요약 → 한국어 번역
if not summary.startswith("Translate Error"):  # 오류가 없을 때만 번역 진행
    korean_summary = translate_to_korean(summary)
else:
    korean_summary = summary  # 요약 실패 시 그대로 반환

print(f"\n 논문 요약 (영어):\n{summary}")
print(f"\n 논문 요약 (한국어 번역):\n{korean_summary}")

 요약 결과 (영어): [{'summary_text': 'The proliferation of novel network security threats further amplifies the suddennessand destructiveness of network threats. Traditional network intrusion detection systems rely on predefined rules or known signatures of attacks. In this paper, we introduce a Time-Space Separable Attention Network (TSSAN) for intrusion detection. TSSAN utilizes depth wise separable convolution and a time-space self-attention.mechanism to effectively extract temporal and spatial features.'}]
번역 API 응답 상태 코드: 200
번역 API 응답 텍스트: [{"translation_text":"새로운 네트워크 보안 위협의 확산은 네트워크 위협의 갑작스럽고 파괴적 인 효과를 더욱 강화시킵니다. 전통적인 네트워크 침입 탐지 시스템은 미리 정의된 규칙이나 공격의 알려진 서명에 의존합니다. 이 논문에서는 침입 탐지 위해 시간 공간 분리 가능한 관심 네트워크 (TSSAN) 를 도입합니다. TSSAN는 깊이별 가능한 분리 가능한 convolution 및 시간 공간 자기주의 메커니즘을 사용하여 시간 및 공간적 특징을 효과적으로 추출합니다."}]

 논문 요약 (영어):
The proliferation of novel network security threats further amplifies the suddennessand destructiveness of network threats. Traditional network intrusion detection syst

In [20]:
# 실행 코드 (로컬 PDF 파일 입력)
def run_model(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # 논문 요약 (영어)
    summary = summarize_text(text)
    # 영어 요약 → 한국어 번역
    if not summary.startswith("Translate Error"):  # 오류가 없을 때만 번역 진행
        korean_summary = translate_to_korean(summary)
    else:
        korean_summary = summary  # 요약 실패 시 그대로 반환

    print(f"\n 논문 요약 (영어):\n{summary}")
    print(f"\n 논문 요약 (한국어 번역):\n{korean_summary}")

pdf_path = "/Users/anchanghun/Downloads/LLM Paper 모음/A Survey of Large Language Models.pdf"

run_model(pdf_path)


 요약 결과 (영어): [{'summary_text': 'Survey of Large Language Models (LLM) is a review of the literature on LLMs, which can be a useful resource for both researchers and engineers. Large language models (LLMs) have been proposed by pre-trained language models over large-scale corpora showing strong capabilities in solving various natural language processing tasks.'}]
번역 API 응답 상태 코드: 200
번역 API 응답 텍스트: [{"translation_text":"대규모 언어 모델 (LLM) 조사 (Survey of Large Language Models, LLM) 는 LLM에 관한 문학의 검토입니다. 이는 연구자와 엔지니어 모두에게 유용한 자원이 될 수 있습니다."}]

 논문 요약 (영어):
Survey of Large Language Models (LLM) is a review of the literature on LLMs, which can be a useful resource for both researchers and engineers. Large language models (LLMs) have been proposed by pre-trained language models over large-scale corpora showing strong capabilities in solving various natural language processing tasks.

 논문 요약 (한국어 번역):
대규모 언어 모델 (LLM) 조사 (Survey of Large Language Models, LLM) 는 LLM에 관한 문학의 검토입니다. 이는 연구자와 엔지니어 모두에게 